In [13]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid
import json

In [27]:
class PET_dataset():
    def __init__(self,part,neck_dir,body_dir,phase, transformer=None, aug=None, aug_factor=0):
        self.neck_dir=neck_dir
        self.body_dir=body_dir
        self.part=part
        self.phase=phase
        self.transformer=transformer
        self.aug=aug
        self.aug_factor=aug_factor
        if(self.part=="body"):
            self.image_files = sorted([fn for fn in os.listdir(self.body_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.body_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        elif(self.part=="neck"):
            self.image_files = sorted([fn for fn in os.listdir(self.neck_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.neck_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        
        self.auged_img_list, self.auged_label_list=self.make_aug_list(self.image_files, self.label_files)
        
    def __getitem__(self,index):
        if(self.aug==None):
            filename, image = self.get_image(self.part, index)
            bboxes, class_ids = self.get_label(self.part, index)

            if(self.transformer):
                transformed_data=self.transformer(image=image, bboxes=bboxes, class_ids=class_ids)
                image = transformed_data['image']
                bboxes = np.array(transformed_data['bboxes'])
                class_ids = np.array(transformed_data['class_ids'])


            target = {}
    #         print(f'bboxes:{bboxes}\nclass_ids:{class_ids}\nlen_bboxes:{len(bboxes)}\nlen_class_ids:{len(class_ids)}')
    #         print(f'filename: {filename}')
            target["boxes"] = torch.Tensor(bboxes).float()
            target["labels"] = torch.Tensor(class_ids).long()

            ###
            bboxes=torch.Tensor(bboxes).float()
            class_ids=torch.Tensor(class_ids).long()
            target = np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1)
            ###
        else:
            image=self.auged_img_list[index][1]
            target=self.auged_label_list[index]
            filename=self.auged_img_list[index][0]
        return image, target, filename
    
    def __len__(self, ):
        length=0
        if(self.aug==None):
            length=len(self.image_files)
        else:
            length=len(self.auged_img_list)
        return length
    
    def make_aug_list(self,ori_image_list,ori_label_files):
        aug_image_list=[]
        aug_label_list=[]
        
        print(f"start making augmented images-- augmented factor:{self.aug_factor}")
        for i in range(len(ori_image_list)):
            filename, ori_image = self.get_image(self.part, i)
            ori_bboxes, ori_class_ids = self.get_label(self.part, i)
            for j in range(self.aug_factor):
                auged_data=self.aug(image=ori_image, bboxes=ori_bboxes, class_ids=ori_class_ids)
                image = auged_data['image']
                bboxes = np.array(auged_data['bboxes'])
                class_ids = np.array(auged_data['class_ids'])
                
                bboxes=torch.Tensor(bboxes).float()
                class_ids=torch.Tensor(class_ids).long()
                
                aug_image_list.append((filename, image))
                aug_label_list.append(np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1))
        
        print(f"total length of augmented images: {len(aug_image_list)}")
        
        return aug_image_list, aug_label_list
        
    
    def get_image(self, part, index): # 이미지 불러오는 함수
        filename = self.image_files[index]
        if(part=="body"):
#             print(f"body called!-> {self.part}")
            image_path = self.body_dir+"/"+self.phase+"/image/"+filename
        elif(part=="neck"):
#             print(f"neck called!-> {self.part}")
            image_path = self.neck_dir+"/"+self.phase+"/image/"+filename
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return filename, image
    
    def get_label(self, part, index): # label (box좌표, class_id) 불러오는 함수
        label_filename=self.label_files[index]
        if(part=="body"):
#             print(f"body label called!-> {self.part}")
            label_path = self.body_dir+"/"+self.phase+"/label/"+label_filename
        elif(part=="neck"):
#             print(f"neck label called!-> {self.part}")
            label_path = self.neck_dir+"/"+self.phase+"/label/"+label_filename
        with open(label_path, 'r') as file:
            labels = file.readlines()
        
        class_ids=[]
        bboxes=[]
        for label in labels:
            label=label.replace("\n", "")
            obj=label.split(' ')[0]
            coor=label.split(' ')[1:]
            obj=int(obj)
            coor=list(map(float, coor))
            class_ids.append(obj)
            bboxes.append(coor)
            
        return bboxes, class_ids
    

In [30]:
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'
IMAGE_SIZE = 448
transformer = A.Compose([
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
            A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
    )
test_dataset=PET_dataset("body" ,neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='test', transformer=transformer, aug=None)

start making augmented images-- augmented factor:0
total length of augmented images: 0


In [33]:
for k in range(len(test_dataset)):
    image=test_dataset[k][0]
    target=test_dataset[k][1]
    filename=test_dataset[k][2]
    print(image.shape, target, filename)

torch.Size([3, 448, 448]) [[0.89257902 0.63102502 0.029324   0.043519   1.        ]
 [0.52333897 0.52550298 0.087971   0.033692   1.        ]
 [0.18073    0.26906899 0.026332   0.033692   1.        ]
 [0.30999401 0.22812399 0.01915    0.028545   1.        ]
 [0.26032299 0.39050099 0.023938   0.023865   1.        ]
 [0.28096899 0.39752001 0.008977   0.012635   0.        ]
 [0.059545   0.30954599 0.016158   0.011699   0.        ]
 [0.090664   0.57955098 0.014961   0.011699   0.        ]
 [0.576002   0.38020599 0.016158   0.010763   0.        ]
 [0.33602601 0.441039   0.013764   0.010763   0.        ]
 [0.87552398 0.49227899 0.017953   0.051474   1.        ]
 [0.61789298 0.46420199 0.035308   0.034628   1.        ]
 [0.96828198 0.153954   0.025135   0.021526   1.        ]] shape1_129.jpg
torch.Size([3, 448, 448]) [[0.63491601 0.26521701 0.023111   0.020595   0.        ]
 [0.55028099 0.114645   0.018738   0.016934   0.        ]
 [0.31167999 0.098169   0.081199   0.043478   1.        ]] sha

In [40]:
part='body'
neck_dir = '/home/host_data/PET_data/Neck'
body_dir = '/home/host_data/PET_data/Body'
phase='test'

ImageToId={"shape4_4.jpg": 0,
  "shape1_129.jpg": 1,
  "shape1_17.jpg": 2,
  "shape1_32.jpg": 3,
  "shape1_53.jpg": 4,
  "shape1_59.jpg": 5,
  "shape1_67.jpg": 6,
  "shape1_81.jpg": 7,
  "shape1_93.jpg": 8,
  "shape3_115.jpg": 9,
  "shape3_131.jpg": 10,
  "shape3_143.jpg": 11,
  "shape3_40.jpg": 12,
  "shape3_64.jpg": 13,
  "shape3_73.jpg": 14,
  "shape3_75.jpg": 15,
  "shape4_27.jpg": 16,
  "shape4_32.jpg": 17,
  "shape4_34.jpg": 18,
  "shape4_35.jpg": 19}
with open('/home/host_data/PET_data/Body/test/test.json', 'r') as f:
        json_data = json.load(f)
        
for k in range(len(test_dataset)):
    image=test_dataset[k][0]
    targets=test_dataset[k][1]
    image_filename=test_dataset[k][2]
    w=image.shape[2]
    h=image.shape[1]
    for target in targets:
        target[0]=target[0]*w
        target[2]=target[2]*w
        target[1]=target[1]*h
        target[3]=target[3]*h
        class_ids.append(obj)
        bboxes.append(coor)
    val={"id": 0,
   "image_id": 0,
   "bbox": [],
   "area": 0,
   "iscrowd": 0,
   "category_id": 0,
   "segmentation": []}
    for i in range(len(targets)):
        val={"id": 0,
       "image_id": 0,
       "bbox": [],
       "area": 0,
       "iscrowd": 0,
       "category_id": 0,
       "segmentation": []}
        val['image_id']=ImageToId[image_filename]
        val['id']=i
        val['area']=targets[i][2]*targets[i][3]
        val['category_id']=int(targets[i][4])
        val['bbox']=targets[i][0:4].tolist()
        json_data['annotations'].append(val)
        with open('/home/host_data/PET_data/Body/test/test.json', 'w', encoding='utf-8') as make_file:
            json.dump(json_data, make_file, indent="\t")
    print(f"image shape:{image.shape}, label:{label_filename}, bbox:{targets[i][0:4]}, class_ids:{targets[i][4]}\n")

    
# with open('/home/host_data/PET_data/Body/test/test.json', 'r') as f:
#         json_data = json.load(f)
# print(json.dumps(json_data['annotations'], indent="\t"))

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[433.79032898  68.97139168  11.26047969   9.64364779], class_ids:1.0

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[139.63263512  43.97971153  36.37715101  19.47814441], class_ids:1.0

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[211.9223671  345.1754837   12.07763207   9.22655964], class_ids:1.0

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[337.45645523 249.60902405   9.15398371   5.55385584], class_ids:0.0

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[431.40250397 251.47269821   8.5043844   15.22841597], class_ids:1.0

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[220.11226082  77.94931412  12.5605762    8.80095994], class_ids:1.0

image shape:torch.Size([3, 448, 448]), label:shape1_129.txt, bbox:[212.25298882  37.55987167  23.76371145  20.71551919], class_ids:1.0

image shape:torch.Size([3, 448, 448]), label:sha